In [1]:
from tqdm import tqdm
from torch.utils.data.dataloader import DataLoader

from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/NLP')

from param_partitioner import *
from dataset import *
from models import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
class Trainer:
    def __init__(self, model, optimiser, dataset, hyp):
        self.model = model
        self.optimiser = optimiser
        self.dataset = dataset
        self.hyp = hyp
        self.test_dataset = None
        self.sample_rate = 100

        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = torch.cuda.current_device()
            self.model = torch.nn.DataParallel(self.model).to(self.device)
    
    def load(self, model_object):
        self.model = model_object
        self.model.load_state_dict(torch.load('state/model'))
        if self.device != 'cpu': self.model = torch.nn.DataParallel(self.model)
        self.model = self.model.to(self.device)

    def sample(self, prompt='', temperature=1):
        if prompt is '':
            x = torch.randint(self.dataset.vocab_size,(1,1))
        else:
            x = [[self.dataset.s2i(s) for s in prompt]]

        x = torch.tensor(x, dtype=torch.long).to(self.device)
        for i in range(100):
            logits = self.model(x)
            logits = logits[:, -1, :] / temperature
            probs = F.softmax(logits, dim=-1)
            w_ix = torch.multinomial(probs, num_samples=1)
            # _, w_ix = torch.topk(probs, k=1, dim=-1)
            x = torch.cat((x, w_ix), dim=1)
            prompt += self.dataset.i2s(int(w_ix))
            if prompt[-1]=='_': break
        return prompt

    def train(self):
        hyp = self.hyp
        def run_epoch(split):
            is_train = split == 'train'
            model.train(is_train)
            loader = DataLoader(self.dataset, shuffle=True, pin_memory=True,
                                batch_size=hyp.batch_size,
                                num_workers=hyp.num_workers)
            losses = []
            pbar = tqdm(enumerate(loader), total=len(loader)) if is_train else enumerate(loader)
            for it, (x, y) in pbar:
                x = x.to(self.device)
                y = y.to(self.device)
                
                with torch.set_grad_enabled(is_train):
                    logits = model(x)
                    loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1))

                    loss = loss.mean()
                    losses.append(loss.item())

                if is_train:
                    if it%self.sample_rate==0 and self.dataset.selfsupervised: 
                        torch.save(model.state_dict(), 'state/model')
                        print( self.sample(self.dataset.rnd_ts()) )
                        
                    model.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), hyp.grad_norm_clip)
                    self.optimiser.step()

                    lr = hyp.learning_rate
                    
                    pbar.set_description(f"epoch {epoch+1} iter {it}: train loss {loss.item():.5f}. lr {lr:e}")

            if not is_train:
                test_loss = float(np.mean(losses))
                logger.info("test loss: %f", test_loss)
                return test_loss

        best_loss = float('inf')
        self.tokens = 0 # counter used for learning rate decay
        for epoch in range(hyp.max_epochs):

            run_epoch('train')
            if self.test_dataset is not None:
                test_loss = run_epoch('test')

            lr = hyp.learning_rate * (hyp.lr_mult ** epoch)
            for param_group in self.optimiser.param_groups:
                param_group['lr'] = lr


class Hyperparameters:
    batch_size = 32
    num_workers = 8
    block_size = 128#50
    h_dim = 256
    grad_norm_clip = 0.5
    learning_rate = 0.003
    lr_mult = 0.9
    max_epochs = 1
    nhead = 4
    dropout = 0.5
    nlayers = 2
    max_T = block_size//2
    betas = (0.9, 0.95)

In [3]:
n = int(10**5)
(tr_xs, tr_ys), (ts_xs, ts_ys) = generate_data(n, difficulty=3, extra=False)

# encode these as batched tensors
# encode (x, y) as tensors add end of stream token
hyp = Hyperparameters()
dataset = CharDataset(tr_xs, tr_ys, ts_xs, ts_ys, hyp.block_size)

model = M3(dataset.vocab_size, hyp)

# initialise model parameters
weight_kwargs = {'Embedding':(nn.init.normal_, [], {'mean':0, 'std':0.02}),
                  'Linear':(nn.init.normal_, [], {'mean':0, 'std':0.02}),
                  'LayerNorm':(nn.init.constant_, [1], {})}
bias_kwargs =   {'Default':(nn.init.constant_, [0], {})}
apply_initialisation(model, (weight_kwargs, bias_kwargs))

# initialise optimiser
conditions = ['weight', 'pos_emb']
combine = {'weights':[[1,1],[1,0]],
            'rest':[[0,0],[0,1]],
            'other': None}
partition = multiple_partition(model, conditions, comb=combine)

for k, v in partition.items(): print('\n'+k+'\n%s'%v)

typ_opt_kwargs = {'weights': {"weight_decay": 0.1}, 
                  'rest':   {"weight_decay": 0.0},
                  'other':   {}}
opt_groups = optimisation_groups(model, partition, typ_opt_kwargs)
optimiser = torch.optim.AdamW(opt_groups, lr=hyp.learning_rate, betas=hyp.betas) 

data has 51092 characters, 33 unique.
M3
Embedding
Sequential
Block
MultiHeadMaskedSelfAttention
Linear
Linear
Linear
Sequential
Linear
GELU
Linear
Dropout
Block
MultiHeadMaskedSelfAttention
Linear
Linear
Linear
Sequential
Linear
GELU
Linear
Dropout
Linear
LayerNorm

weights
['blocks.1.mlp.0.weight', 'blocks.0.mlp.2.weight', 'prj.weight', 'blocks.0.atn.val.weight', 'blocks.0.atn.qry.weight', 'blocks.1.atn.key.weight', 'blocks.0.mlp.0.weight', 'blocks.1.atn.qry.weight', 'blocks.1.atn.val.weight', 'blocks.1.mlp.2.weight', 'tok_emb.weight', 'blocks.0.atn.key.weight', 'ln.weight']

rest
['blocks.0.mlp.0.bias', 'blocks.1.atn.val.bias', 'blocks.1.atn.qry.bias', 'blocks.1.atn.key.bias', 'prj.bias', 'blocks.1.mlp.0.bias', 'blocks.0.mlp.2.bias', 'ln.bias', 'blocks.0.atn.qry.bias', 'blocks.0.atn.key.bias', 'blocks.1.mlp.2.bias', 'blocks.0.atn.val.bias', 'pos_emb']

other
[]


In [4]:
trainer = Trainer(model, optimiser, dataset, hyp)
trainer.train()
# trainer.load(M3(dataset.vocab_size, hyp))

# # then do supervised fine tuning and only train the extended part of the model
# trainer.dataset.selfsupervised = False

# '''
# could feed in x with no mask
# then pull out next value 10 times, compare to y
# '''

# extended_model = Extention(model, dataset.vocab_size, hyp)

# hyp.max_epochs = 1
# trainer = Trainer(extended_model, dataset, hyp)# turn off grad for end params!
# trainer.train()

# for (x, y) in trainer.dataset:
#     print(''.join([trainer.dataset.i2s(i) for i in x]), end='   ')
#     print(''.join([trainer.dataset.i2s(i) for i in y]))
#     yhat = trainer.model(x.unsqueeze(0))[0]
#     _, w_ix = torch.topk(F.softmax(yhat, dim=-1), k=1, dim=-1)
#     print(''.join([trainer.dataset.i2s(i) for i in w_ix]), end='\n\n')


epoch 1 iter 6: train loss 1.86064. lr 3.000000e-03:   0%|          | 5/37726 [00:00<1:26:00,  7.31it/s]

multiply 7 by 10lsu 0pev7e36-t6mco7n5u4p4.p5:mpm49e14_


epoch 1 iter 108: train loss 1.39333. lr 3.000000e-03:   0%|          | 109/37726 [00:02<16:19, 38.40it/s]

multiply 324 by -443 218 and:309_


epoch 1 iter 207: train loss 1.09336. lr 3.000000e-03:   1%|          | 204/37726 [00:05<17:10, 36.40it/s]

...

epoch 1 iter 37607: train loss 0.72092. lr 3.000000e-03: 100%|█████████▉| 37605/37726 [15:36<00:03, 35.86it/s]

subtract 375 from -34:-499_


epoch 1 iter 37707: train loss 0.72372. lr 3.000000e-03: 100%|█████████▉| 37708/37726 [15:39<00:00, 36.99it/s]

add 17 and 32:69_


epoch 1 iter 37725: train loss 0.72390. lr 3.000000e-03: 100%|██████████| 37726/37726 [15:40<00:00, 40.13it/s]


In [6]:
(xs, ys), _ = generate_data(30, difficulty=3, extra=False)
for x,y in zip(xs, ys): 
    yhat = trainer.sample(x+':')[:-1]
    i = float(yhat.split(':')[-1])
    print(('✅\t\t' if float(y)==i else '❌ err: '+str(float(y)-i)+'\t')+yhat)

✅		divide -33 by 5:-6.6000
✅		multiply 1 by 43:43
✅		add 34 and 454:488
✅		add 3 and 347:350
✅		add 3 and -456:-453
✅		subtract -421 from -4:417
✅		subtract 3 from -49:-52
❌ err: 203.0	add 223 and 2:22
❌ err: -8.0	add 49 and 2:59
✅		add 27 and -4:23
❌ err: 2.0	add 4 and -2:-0
❌ err: 80.0	multiply 21 by -49:-1109
✅		add 20 and -4:16
✅		multiply -38 by -3:114
✅		divide 2 by 459:0.0044
✅		multiply -7 by -2:14
❌ err: 0.2065999999999999	divide 329 by -24:-13.9149
✅		add 206 and 1:207
✅		add -3 and -16:-19
✅		divide -2 by 23:-0.0870
❌ err: -40.0	multiply 14 by -196:-2704
✅		subtract 2 from -50:-52
❌ err: -30.0	multiply -271 by 4:-1054
✅		divide 43 by 1:43.0000
